In [25]:
from PIL import Image
import glob
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [26]:
labels = pd.read_csv("Xray_TeethLabels_Simple.csv",index_col=0)
Ylabels = labels[5:]

metaY = labels[:3]

X = {}
Y = {}
count1 = 0
count2 = 0
count3 = 0
for picFilename in glob.glob("Processing2/processed/*"):
    patNumber =  picFilename.split('/')[2].split('_')[0]
    toothNumber = picFilename.split('/')[2].split('_')[1].split('.')[0]
    im = Image.open(picFilename)

    if int(metaY.loc['Age',patNumber]) < 25:
        count1+=1
        if 'Age1' in X.keys():
            X['Age1'][patNumber + '_' + toothNumber] = np.array(im).flatten()
            Y['Age1'].append(1 if Ylabels.loc[toothNumber,patNumber] == 'Yes' else 0)
        else:
            X['Age1'] = pd.DataFrame()
            X['Age1'][patNumber + '_' + toothNumber] = np.array(im).flatten()
            Y['Age1'] = [1 if Ylabels.loc[toothNumber,patNumber] == 'Yes' else 0]
    
    elif int(metaY.loc['Age',patNumber]) < 40:
        count2+=1
        if 'Age2' in X.keys():
            X['Age2'][patNumber + '_' + toothNumber] = np.array(im).flatten()
            Y['Age2'].append(1 if Ylabels.loc[toothNumber,patNumber] == 'Yes' else 0)
        else:
            X['Age2'] = pd.DataFrame()
            X['Age2'][patNumber + '_' + toothNumber] = np.array(im).flatten()
            Y['Age2'] = [1 if Ylabels.loc[toothNumber,patNumber] == 'Yes' else 0]
    
    else:
        count3+=1
        if 'Age3' in X.keys():
            X['Age3'][patNumber + '_' + toothNumber] = np.array(im).flatten()
            Y['Age3'].append(1 if Ylabels.loc[toothNumber,patNumber] == 'Yes' else 0)
        else:
            X['Age3'] = pd.DataFrame()
            X['Age3'][patNumber + '_' + toothNumber] = np.array(im).flatten()
            Y['Age3'] = [1 if Ylabels.loc[toothNumber,patNumber] == 'Yes' else 0]

In [27]:
print(count1)
print(count2)
print(count3)

80
60
84


In [28]:
np.random.seed(6)
def svc_param_selection(X, y, nfolds):
    C = [0.1, 1, 10, 100, 1000] #try different values for C
    param_grid = {'C': C}
    grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

print ("The best value for C is:",svc_param_selection(ageX.T.to_numpy(), np.array(ageY), 2))

The best value for C is: {'C': 0.1}


In [29]:
for age in X.keys():
    ageX = X[age]
    ageY = Y[age]

    X_train, X_test, y_train, y_test = train_test_split(ageX.T.to_numpy(), np.array(ageY), test_size=0.20)
    
    best_c = svc_param_selection(ageX.T.to_numpy(), np.array(ageY), 2)
    print ("The best value for C is:",best_c)

    print(f"SVM modle for age: {age}")
    model = SVC(C=best_c['C'], kernel='rbf')
    model.fit(X_train, y_train)
    print(model)
    # make predictions
    expected = y_test
    predicted = model.predict(X_test)
    # summarize the fit of the model
    print(metrics.classification_report(expected, predicted))
    print(metrics.confusion_matrix(expected, predicted))

    # print("KNN Accuracy: {}%".format(stat.mean(neighAcc)*100))

The best value for C is: {'C': 10}
SVM modle for age: Age2
SVC(C=10)
              precision    recall  f1-score   support

           0       0.50      0.29      0.36         7
           1       0.38      0.60      0.46         5

    accuracy                           0.42        12
   macro avg       0.44      0.44      0.41        12
weighted avg       0.45      0.42      0.40        12

[[2 5]
 [2 3]]
The best value for C is: {'C': 100}
SVM modle for age: Age1
SVC(C=100)
              precision    recall  f1-score   support

           0       0.71      0.71      0.71         7
           1       0.78      0.78      0.78         9

    accuracy                           0.75        16
   macro avg       0.75      0.75      0.75        16
weighted avg       0.75      0.75      0.75        16

[[5 2]
 [2 7]]
The best value for C is: {'C': 0.1}
SVM modle for age: Age3
SVC(C=0.1)
              precision    recall  f1-score   support

           0       0.53      1.00      0.69       

/usr/local/anaconda3/envs/opencv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/envs/opencv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/envs/opencv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
for age in X.keys():
    ageX = X[age]
    ageY = Y[age]

    X_train, X_test, y_train, y_test = train_test_split(ageX.T.to_numpy(), np.array(ageY), test_size=0.20)
    best_c = svc_param_selection(ageX.T.to_numpy(), np.array(ageY), 2)
    
    print(f"SVM modle for age: {age}")
    model = SVC(C=best_c['C'], kernel='linear')
    model.fit(X_train, y_train)
    print(model)
    # make predictions
    expected = y_test
    predicted = model.predict(X_test)
    # summarize the fit of the model
    print(metrics.classification_report(expected, predicted))
    print(metrics.confusion_matrix(expected, predicted))

    # print("KNN Accuracy: {}%".format(stat.mean(neighAcc)*100))

SVM modle for age: Age2
SVC(C=10, kernel='linear')
              precision    recall  f1-score   support

           0       0.43      0.75      0.55         4
           1       0.80      0.50      0.62         8

    accuracy                           0.58        12
   macro avg       0.61      0.62      0.58        12
weighted avg       0.68      0.58      0.59        12

[[3 1]
 [4 4]]
SVM modle for age: Age1
SVC(C=100, kernel='linear')
              precision    recall  f1-score   support

           0       0.50      0.50      0.50         6
           1       0.70      0.70      0.70        10

    accuracy                           0.62        16
   macro avg       0.60      0.60      0.60        16
weighted avg       0.62      0.62      0.62        16

[[3 3]
 [3 7]]
SVM modle for age: Age3
SVC(C=0.1, kernel='linear')
              precision    recall  f1-score   support

           0       0.77      0.77      0.77        13
           1       0.25      0.25      0.25         